In [1]:
import findspark
findspark.init()
findspark.find()
from pyspark.sql import SparkSession


spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [2]:
from pyspark.sql.functions import col
from pyspark.sql import functions as f
from pyspark.sql.types import IntegerType, StringType, ArrayType
from pyspark.sql.functions import monotonically_increasing_id

In [3]:
df = spark.read.load('data/foodmart.csv', format="csv", header=True, delimiter=",")
df.show(5)

+------------+---------+-------+--------------+------+---------+----+-------+-------+------------+-----------------+------+------+-----+---------+---------------+-----+--------+------+-------------+------------------+-----------+-------+-----------+--------------+--------+----------+-----------+-----------+----+------+-----------+----------+----+-----------------+---------------+------------+-------------+----------+--------------+-----------------+---+---------+----------+--------------+--------+---------+---------+---+-----+-----+----------+----+----+---------+-------+------------+----+-------+-----------+--------+------------+-----------+-----+-------------+----------------+-----+----------------+-------+---------+------------+-------------+-------------+---------+--------+----+--------+------+------------+-------+---------+------+------------+----+----+----------+------+-------+----------------+-----+----------+----+--------------+-----+------------+----+---------+-------+----+----

### Biến đổi dữ liệu input thành dataframe gồm 2 cột: cột ID tăng dần, cột Items chứa tên các item mua cách nhau bởi dấu phẩy

In [39]:
from pyspark.sql.types import IntegerType, StringType

def func(row):
  arr = []
  for i in range(len(row)):
    if row[i] == '1':
      arr.append(cols[i])
  return ",".join(arr)

cols = df.columns
bought_list = f.udf(lambda row: func(row), StringType())  # udf: user define function

items_df = df.withColumn("Items", bought_list(f.struct([x for x in df.columns])))
# items_df = df.withColumn("Items", bought_list(f.struct([df[x] for x in df.columns])))
items_df.select('Items').show(5, truncate=False)

+------------------------------------------------------+
|Items                                                 |
+------------------------------------------------------+
|Acetominifen,Cheese,Home Magazines,Shampoo            |
|Acetominifen,Cheese,Hard Candy,Milk,Pot Scrubbers,Rice|
|Coffee,Deli Salads                                    |
|Eggs,Gum,Milk,Soup                                    |
|Cheese,Dried Fruit,Frozen Chicken,Plastic Utensils    |
+------------------------------------------------------+
only showing top 5 rows



In [31]:
processed_df = items_df.withColumn("ID", monotonically_increasing_id() + 1).select('ID', 'Items')
processed_df.show(truncate=False)

+---+---------------------------------------------------------------+
|ID |Items                                                          |
+---+---------------------------------------------------------------+
|1  |Acetominifen,Cheese,Home Magazines,Shampoo                     |
|2  |Acetominifen,Cheese,Hard Candy,Milk,Pot Scrubbers,Rice         |
|3  |Coffee,Deli Salads                                             |
|4  |Eggs,Gum,Milk,Soup                                             |
|5  |Cheese,Dried Fruit,Frozen Chicken,Plastic Utensils             |
|6  |Shampoo                                                        |
|7  |Milk,Paper Wipes,Waffles                                       |
|8  |Donuts,Dried Fruit,Frozen Chicken                              |
|9  |Cooking Oil,Hamburger,Maps,Popsicles                           |
|10 |Cheese,Cooking Oil,Dips,Preserves,TV Dinner                    |
|11 |Nasal Sprays                                                   |
|12 |Auto Magazines,

In [9]:
#nhap
f.struct([df[x] for x in df.columns])

Column<'struct(Acetominifen, Anchovies, Aspirin, Auto Magazines, Bagels, Batteries, Beer, Bologna, Candles, Canned Fruit, Canned Vegetables, Cereal, Cheese, Chips, Chocolate, Chocolate Candy, Clams, Cleaners, Coffee, Cold Remedies, Computer Magazines, Conditioner, Cookies, Cooking Oil, Cottage Cheese, Crackers, Deli Meats, Deli Salads, Deodorizers, Dips, Donuts, Dried Fruit, Dried Meat, Eggs, Fashion Magazines, Flavored Drinks, French Fries, Fresh Chicken, Fresh Fish, Frozen Chicken, Frozen Vegetables, Gum, Hamburger, Hard Candy, Home Magazines, Hot Dogs, Ibuprofen, Ice Cream, Jam, Jelly, Juice, Lightbulbs, Maps, Milk, Mouthwash, Muffins, Nasal Sprays, Nuts, Oysters, Pancake Mix, Pancakes, Paper Dishes, Paper Wipes, Pasta, Peanut Butter, Personal Hygiene, Pizza, Plastic Utensils, Popcorn, Popsicles, Pot Cleaners, Pot Scrubbers, Pots and Pans, Preserves, Pretzels, Rice, Sardines, Sauces, Screwdrivers, Shampoo, Shellfish, Shrimp, Sliced Bread, Soda, Soup, Sour Cream, Spices, Sponges, Spo

In [40]:
#nhap
f.struct([x for x in df.columns])

Column<'struct(Acetominifen, Anchovies, Aspirin, Auto Magazines, Bagels, Batteries, Beer, Bologna, Candles, Canned Fruit, Canned Vegetables, Cereal, Cheese, Chips, Chocolate, Chocolate Candy, Clams, Cleaners, Coffee, Cold Remedies, Computer Magazines, Conditioner, Cookies, Cooking Oil, Cottage Cheese, Crackers, Deli Meats, Deli Salads, Deodorizers, Dips, Donuts, Dried Fruit, Dried Meat, Eggs, Fashion Magazines, Flavored Drinks, French Fries, Fresh Chicken, Fresh Fish, Frozen Chicken, Frozen Vegetables, Gum, Hamburger, Hard Candy, Home Magazines, Hot Dogs, Ibuprofen, Ice Cream, Jam, Jelly, Juice, Lightbulbs, Maps, Milk, Mouthwash, Muffins, Nasal Sprays, Nuts, Oysters, Pancake Mix, Pancakes, Paper Dishes, Paper Wipes, Pasta, Peanut Butter, Personal Hygiene, Pizza, Plastic Utensils, Popcorn, Popsicles, Pot Cleaners, Pot Scrubbers, Pots and Pans, Preserves, Pretzels, Rice, Sardines, Sauces, Screwdrivers, Shampoo, Shellfish, Shrimp, Sliced Bread, Soda, Soup, Sour Cream, Spices, Sponges, Spo

In [23]:
#nhap
cols = df.columns
tmp = df.collect()[0]  # get first row
[cols[i] for i in range(len(tmp)) if tmp[i] == '1']

['Acetominifen', 'Cheese', 'Home Magazines', 'Shampoo']

### Biến đổi dữ liệu input thành dataframe gồm 2 cột: cột ID tăng dần, cột Items là array chứa tên các item mua

In [34]:
from pyspark.sql.types import IntegerType, StringType, ArrayType

def func(row):
  arr = []
  for i in range(len(row)):
    if row[i] == '1':
      arr.append(cols[i])
  return arr

cols = df.columns
bought_list = f.udf(lambda row: func(row), ArrayType(StringType()))

items_df = df.withColumn("Items", bought_list(f.struct([df[x] for x in df.columns])))
items_df.select('Items').show(5, truncate=False)

+-------------------------------------------------------------+
|Items                                                        |
+-------------------------------------------------------------+
|[Acetominifen, Cheese, Home Magazines, Shampoo]              |
|[Acetominifen, Cheese, Hard Candy, Milk, Pot Scrubbers, Rice]|
|[Coffee, Deli Salads]                                        |
|[Eggs, Gum, Milk, Soup]                                      |
|[Cheese, Dried Fruit, Frozen Chicken, Plastic Utensils]      |
+-------------------------------------------------------------+
only showing top 5 rows



In [35]:
from pyspark.sql.functions import monotonically_increasing_id
processed_df = items_df.withColumn("ID", monotonically_increasing_id() + 1).select('ID', 'Items')
processed_df.show(truncate=False)

+---+---------------------------------------------------------------------+
|ID |Items                                                                |
+---+---------------------------------------------------------------------+
|1  |[Acetominifen, Cheese, Home Magazines, Shampoo]                      |
|2  |[Acetominifen, Cheese, Hard Candy, Milk, Pot Scrubbers, Rice]        |
|3  |[Coffee, Deli Salads]                                                |
|4  |[Eggs, Gum, Milk, Soup]                                              |
|5  |[Cheese, Dried Fruit, Frozen Chicken, Plastic Utensils]              |
|6  |[Shampoo]                                                            |
|7  |[Milk, Paper Wipes, Waffles]                                         |
|8  |[Donuts, Dried Fruit, Frozen Chicken]                                |
|9  |[Cooking Oil, Hamburger, Maps, Popsicles]                            |
|10 |[Cheese, Cooking Oil, Dips, Preserves, TV Dinner]                    |
|11 |[Nasal 

In [36]:
from pyspark.ml.fpm import FPGrowth
fpGrowth = FPGrowth(itemsCol="Items", minSupport=0.1, minConfidence=0.9)
model = fpGrowth.fit(processed_df)
# get the frequent itemsets
patternsDF = model.freqItemsets
# get the association rules
rulesDF = model.associationRules

D:\Spark\spark-3.2.0-bin-hadoop3.2\python\pyspark\sql\context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [37]:
patternsDF.show(truncate=False)

+-------------+----+
|items        |freq|
+-------------+----+
|[Dried Fruit]|256 |
|[Soup]       |280 |
|[Cookies]    |238 |
|[Cheese]     |285 |
+-------------+----+



In [38]:
rulesDF.show(truncate=False)

+----------+----------+----------+----+-------+
|antecedent|consequent|confidence|lift|support|
+----------+----------+----------+----+-------+
+----------+----------+----------+----+-------+

